In [1]:
import numpy as np
import GOL

In [2]:
#generate training data that is sequential
data_X = []
data_X.append(np.random.randint(0, 2, size = (7, 7)))
data_X.append(GOL.returnNextGen(data_X[0]))

In [3]:
def generateTrainingData(numArrays, listToAddTo):
    for i in range(0, numArrays):
        setToAppend = []
        randomFrame = np.random.randint(0, 2, size = (7, 7))
        setToAppend.append(randomFrame)
        setToAppend.append(GOL.returnNextGen(randomFrame))
        setToAppend.append(0)
        listToAddTo.append(setToAppend)

In [4]:
trainingList = []
generateTrainingData(3000, trainingList)

In [5]:
#generate training data that is not sequential
def generateNonSequentialTrainingData(numArrays, listToAddTo):
    for i in range(0, numArrays):
        setToAppend = []
        randomFrame = np.random.randint(0, 2, size = (7, 7))
        randomFrame2 = np.random.randint(0, 2, size = (7, 7))
        while np.array_equal(randomFrame, randomFrame2):
            randomFrame2 = np.random.randint(0, 2, size = (7, 7))
        setToAppend.append(randomFrame)
        setToAppend.append(randomFrame2)
        setToAppend.append(1)
        listToAddTo.append(setToAppend)


In [6]:
generateNonSequentialTrainingData(3000, trainingList)

In [7]:
import random

random.shuffle(trainingList)

In [8]:
values = [x[2] for x in trainingList]

In [9]:
trainingData_X = [x[:2] for x in trainingList]

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(trainingData_X, values, test_size = 0.2, random_state = 42)

In [11]:
X_train = np.asarray(X_train)

from keras.models import Sequential, Input, Model

In [12]:
toInput = X_train.reshape(4800, 98)
toOutputTrain = np.asarray(y_train).reshape(4800,)

In [13]:
import keras
from keras.models import Sequential, Input, Model
from keras.layers import Dense, LSTM
from keras import losses

model = Sequential()
model.add(Dense(198, activation = 'relu'))
model.add(Dense(392, activation = 'relu'))
model.add(Dense(1,  activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = losses.binary_crossentropy, metrics = ['accuracy'])
GOLFit = model.fit(toInput, toOutputTrain, validation_split = 0.2, epochs = 50)

Using TensorFlow backend.


Train on 3840 samples, validate on 960 samples
Epoch 1/50
3840/3840 [==============================] - 0s 125us/step - loss: 0.4185 - accuracy: 0.7982 - val_loss: 0.2979 - val_accuracy: 0.8813
Epoch 2/50
3840/3840 [==============================] - 0s 84us/step - loss: 0.2682 - accuracy: 0.8917 - val_loss: 0.2407 - val_accuracy: 0.9094
Epoch 3/50
3840/3840 [==============================] - 0s 81us/step - loss: 0.1728 - accuracy: 0.9352 - val_loss: 0.2154 - val_accuracy: 0.9083
Epoch 4/50
3840/3840 [==============================] - 0s 81us/step - loss: 0.1243 - accuracy: 0.9560 - val_loss: 0.2360 - val_accuracy: 0.9031
Epoch 5/50
3840/3840 [==============================] - 0s 83us/step - loss: 0.0622 - accuracy: 0.9810 - val_loss: 0.2242 - val_accuracy: 0.9250
Epoch 6/50
3840/3840 [==============================] - 0s 119us/step - loss: 0.0386 - accuracy: 0.9885 - val_loss: 0.1917 - val_accuracy: 0.9302
Epoch 7/50
3840/3840 [==============================] - 0s 121us/step - loss: 0.0

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 198)               19602     
_________________________________________________________________
dense_2 (Dense)              (None, 392)               78008     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 393       
Total params: 98,003
Trainable params: 98,003
Non-trainable params: 0
_________________________________________________________________


In [15]:
X_test = np.asarray(X_test)

In [16]:
toTest = X_test.reshape(1200, 98)
toOutputTest = np.asarray(y_test).reshape(1200,)

In [17]:
test_loss, test_acc = model.evaluate(toTest, toOutputTest)
print('Test accuracy:', test_acc)

1200/1200 [==============================] - 0s 72us/step
Test accuracy: 0.9424999952316284


In [18]:
y_score_train = model.predict_classes(toInput)
y_score_test = model.predict_classes(toTest)

In [19]:
train_label_score = toOutputTrain
test_label_score = toOutputTest

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

train_accuracy = accuracy_score(train_label_score, y_score_train)
print("Training Accuracy: %f percent" %(train_accuracy*100))
train_precision = precision_score(train_label_score, y_score_train)
print("Training Precision: %f percent" %(train_precision * 100))
train_recall = recall_score(train_label_score, y_score_train)
print("Training Recall: %f percent" %(train_recall * 100))
test_accuracy = accuracy_score(test_label_score, y_score_test)
print("Testing accuracy: %f percent" %(test_accuracy * 100))
test_precision = precision_score(test_label_score, y_score_test)
print("Test precision: %f percent" %(test_precision * 100))
test_recall = recall_score(test_label_score, y_score_test)
print("Test recall: %f percent" %(test_recall * 100))

Training Accuracy: 98.812500 percent
Training Precision: 99.003322 percent
Training Recall: 98.634671 percent
Testing accuracy: 94.250000 percent
Test precision: 93.559322 percent
Test recall: 94.682676 percent


In [21]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [22]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk
